In [23]:
from __future__ import print_function    # (at top of module)
import warnings
warnings.filterwarnings('ignore')
from spotipy.oauth2 import SpotifyClientCredentials
import json
import spotipy
import time
import csv
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 10]
from matplotlib.pyplot import figure
import math
import seaborn as sns
import io, os, sys, types

In [24]:
# Read the data from the file
data = pd.read_csv('Data/data_500_entries_youtube.csv')
#data = pd.read_csv('Data/data_3000_entries_youtube.csv')

print("Number of entries in original data: " + str(len(data.index)))
data.head()

Number of entries in original data: 570


,song_id,song_title,artist,popularity,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration,loudness,valence,mode,youtube_view_count,youtube_video_title
0,spotify:track:5ygDXis42ncn6kYG14lEVG,Baby Shark,[Pinkfong],77,0.840,0.3410,115.062,0.2270,0.2450,0.000000,4,0.825,7,96333,-3.651,0.520,1,1956582159,Baby Shark Dance | Sing and Dance! | Animal So...
1,spotify:track:7fa9MBXhVfQ8P8Df9OEbD8,Girls Like You (feat. Cardi B),"[Maroon 5, Cardi B]",86,0.541,0.1300,124.959,0.0505,0.5680,0.000000,4,0.851,0,235545,-6.825,0.448,1,1300452389,Maroon 5 - Girls Like You ft. Cardi B
2,spotify:track:6De0lHrwBfPfrhorm9q1Xl,Me Rehúso,[Danny Ocean],83,0.804,0.0494,104.823,0.0677,0.0231,0.000000,4,0.744,1,205715,-6.327,0.426,1,1229501096,Danny Ocean - Me Rehúso (Official Audio)
3,spotify:track:1j6xOGusnyXq3l6IryKF3G,Déjala Que Vuelva (feat. Manuel Turizo),"[Piso 21, Manuel Turizo]",74,0.788,0.0753,170.019,0.0785,0.0482,0.000000,4,0.681,1,220117,-4.323,0.839,1,1216075058,Piso 21 - Déjala Que Vuelva (feat. Manuel Turi...
4,spotify:track:2ijef6ni2amuunRoKTlgww,Sin Pijama,"[Becky G, Natti Natasha]",90,0.745,0.1040,94.014,0.0464,0.3540,0.000029,4,0.791,11,188560,-3.695,0.820,0,1071141995,Becky G Natti Natasha - Sin Pijama (Video Ofic...


In [25]:
# Drop unnecessary text columns or columns used for labeling from data
if 'total_no_streams' in data.columns:
    data.drop(['song_id', 'song_title', 'artist', 'popularity', 'youtube_view_count', 'youtube_video_title', 'total_no_streams'], 1, inplace=True)
else:
    data.drop(['song_id', 'song_title', 'artist', 'youtube_view_count', 'youtube_video_title'], 1, inplace=True) 

In [26]:
# X will be our examples and y will be our labels
X = data.drop('popularity', axis=1)
y = data['popularity']

# Sanity checks
print("Number of entries in actual data: " + str(len(X.index)))
print("Number of entries in label data: " + str(len(y.index)))
X.head()

Number of entries in actual data: 570
Number of entries in label data: 570


,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration,loudness,valence,mode
0,0.840,0.3410,115.062,0.2270,0.2450,0.000000,4,0.825,7,96333,-3.651,0.520,1
1,0.541,0.1300,124.959,0.0505,0.5680,0.000000,4,0.851,0,235545,-6.825,0.448,1
2,0.804,0.0494,104.823,0.0677,0.0231,0.000000,4,0.744,1,205715,-6.327,0.426,1
3,0.788,0.0753,170.019,0.0785,0.0482,0.000000,4,0.681,1,220117,-4.323,0.839,1
4,0.745,0.1040,94.014,0.0464,0.3540,0.000029,4,0.791,11,188560,-3.695,0.820,0


In [27]:
# We split the data into :
# X_cv - will be used for cross validation
# X_test - will be used to test the model after the CV, on data that is
#          unseen -> test how well the model will do in the real world
from sklearn.model_selection import train_test_split
X_cv, X_test, y_cv, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

print("Items in CV data set : ", str(len(X_cv.index)))
print("Items in testing data set: ", str(len(X_test.index)))
X_cv.head()

Items in CV data set :  456
Items in testing data set:  114


,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration,loudness,valence,mode
306,0.699,0.123,76.854,0.1430,0.01170,0.093000,4,0.430,1,192941,-5.892,0.473,1
564,0.500,0.343,183.964,0.1880,0.05640,0.000007,4,0.653,10,204080,-9.390,0.593,0
197,0.727,0.115,120.003,0.0324,0.00953,0.000000,4,0.670,1,279240,-7.679,0.405,0
376,0.622,0.099,148.040,0.2030,0.09650,0.000000,4,0.740,1,286280,-6.297,0.393,1
244,0.630,0.207,126.006,0.0763,0.13600,0.000005,4,0.797,5,206212,-7.601,0.761,1


In [28]:
# Those are the columns of features that will be used for prediction
# in our data
COLUMNS_TO_SCALE = ["energy", "liveness", "tempo", 
                    "speechiness", "acousticness", "instrumentalness", 
                    "time_signature", "danceability", "key", 
                    "duration", "loudness", "valence", "mode"]

In [29]:
from sklearn import preprocessing

# We initialize a scaler
scaler = preprocessing.StandardScaler()

# Fit the scaler to our data
scaler.fit(X_cv)

# Scale both our data sets and keep them in pandas DataFrames
X_cv = pd.DataFrame(scaler.transform(X_cv), columns=COLUMNS_TO_SCALE)
X_test = pd.DataFrame(scaler.transform(X_test), columns=COLUMNS_TO_SCALE)

In [30]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import SVR

In [55]:
reg = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=5)
reg.fit(X,y)
reg.score(X, y)

0.7730296527635099

In [56]:
test_example = np.array([[0.317, 0.0881, 140.048, 0.334, 0.592, 0, 4, 0.778, 1, 178627, -10.732, 0.327, 0]])
reg.predict(test_example)

array([76.98550633])